In [1]:
from pandas import DataFrame
import pandas as pd

In [3]:
df1 = pd.read_csv("/home/jonathan/tesis/10mo_avance/data/timeseries_o.csv", sep=',',index_col=0, parse_dates=True)

In [11]:
# df1.tail()

In [4]:
class RouteTrip: # Mi Clase
    __COEF_BASENAME = PassengersDataLoader.BASEPATH + "%s_coef.csv" #data/pt_data/%s_coef.csv

    def __init__(self,tripsloader,ptdata): #Mi constructor - construye un nuevo objeto, self parametro obligatorio
        self.tripsLoader = tripsloader  #propiedad del objeto que recibe el parámetro tripsloader
        self.ptdata = ptdata #propiedad del objeto para elegir el archivo con la clase PassengersDataLoader

        self.routes = tripsloader.routesLoader.routes.set_index("stop_id") # routesloader es la clase, routes es la propiedad del objetos

        self.emtproperties = self.compute_emt_lines_properties() #propiedad del objeto

NameError: name 'PassengersDataLoader' is not defined

In [ ]:
#linea 604 del archivo processor.py
#tripsloader = TripsLoader(verbose = True)

#linea 6 del archivo data_loader.py
# class TripsLoader:
#     BASEPATH = "data/trips_data/"
#     ORIGIN_FILENAME = "timeseries_o.csv"
#     DEST_FILENAME = "timeseries_d.csv"

#linea 605 del archivo processor.py
# ptdata = PassengersDataLoader()

#linea 84 del archivo data_loader.py
# class PassengersDataLoader:
#     BASEPATH = "data/pt_data/"
#     RENFE_MONTHLY_DATA_PATH = BASEPATH + "renfe_monthly_data.csv"
#     RENFE_USERS_BYSTOP = BASEPATH + "up_down_bystop.csv"
#     RENFE_LINEFREQ_FILENAME = BASEPATH + 'renfe_line_freq.csv'
#     METRO_LINEFREQ_FILENAME = BASEPATH + 'metro_line_freq.csv'
#     METRO_DAILY_DATA = BASEPATH + 'metro_daily_data.csv'
#     METRO_USERS_BYSTOP = BASEPATH + 'metro_up_down_bystop.csv'
#     EMT_COEF = BASEPATH + 'emt_coef.csv'
#     EMT_MONTHLY_DATA = BASEPATH + 'emt_monthly_data.csv'
#     EMT_LINEFREQ_FILENAME = BASEPATH + 'emt_line_freq.csv'

#linea 196 del archivo data_loader.py
# class RoutesLoader:
#     BASEPATH = "data/routes/"
#     ROUTES_FILENAME = "routes.csv"

In [ ]:
def get_users_renfe(self, timeseries_o,timeseries_d,max_precision = True):
        # Es el df routes con index "stops_id" 241 filas y 7 columnas, es decir que está filtrado y la columna index es la 8va columna
        # renfe_users_bystop contiene up_down_bystop.csv 
        renfe_stops = self.routes.loc[self.ptdata.renfe_users_bystop[self.ptdata.renfe_users_bystop.stop_id.notna()].stop_id]
        renfe_stops = renfe_stops[~renfe_stops.index.duplicated(keep='first')]

        subtimeseries_o = timeseries_o[renfe_stops.stop_district.unique()]
        subtimeseries_o = subtimeseries_o.between_time("6:00", "0:00")

        subtimeseries_d = timeseries_d[renfe_stops.stop_district.unique()]
        subtimeseries_d = subtimeseries_d.between_time("6:00", "0:00")

        Td = self.ptdata.renfe_monthly_data

        up,down = self.compute_users_method1(renfe_stops,subtimeseries_o,subtimeseries_d,Td,'cercanias')
        if max_precision:
            up2,down2 = self.compute_users_method2(renfe_stops,subtimeseries_o,subtimeseries_d,self.ptdata.renfe_users_bystop,Td,'cercanias')
            up = (up + up2)/2
            down = (down + down2)/2

        return up,down

In [ ]:
def compute_users_method2(self,stops,subtimeseries_o,subtimeseries_d,usersbystop,Td,service):        
        #service Apenas entrando en esta función service ya es "cercanias"
        #subtimeseries_o,subtimeseries_d columns filtradas del df timeseries_o mediante los distritos presentes en "routes", cuidado que en routes se repiten los distritos, el index son las fechas
        #Td es el df llamado "0710200", Daniel tiene los datos desde 2013 hasta junio 2021, index las fechas '2013-01'
        beta = usersbystop # representa el df llamado "up_down_bystop.csv" creado por daniel mediante los GTFS, no tiene index
        stops = stops["stop_district"] #Selecciona la columna "stop_district" del df creado por daniel "routes", el index df stops es el que tiene esto: par_5_142 (stop_id)

        beta = beta[beta.stop_id.notna()] # Descartamos los registros que no tengan stop_id del df usersbystop
        beta.up = beta.up / beta.up.sum() #up es una columna, beta.up es un df de una columna antes y despues de normalizar 
        beta.down = beta.down / beta.down.sum() #down se una columna

        up = pd.DataFrame({}) #Creamos df vacios
        down = pd.DataFrame({})
        if service == "cercanias": #Entra a la primera
            for time_period in Td.index:
                #subtimeseries_op es un df vacio cuando la condición booleana no se cumple
                #subtimeseries_op en cada ciclo for obtiene los datos desplazamientos por mes (Ej: en un mes corresponde a feb otro mes a marzo junto con todos los distritos y desplazamientos(matriz)) 
                subtimeseries_op = subtimeseries_o[(subtimeseries_o.index.month == time_period.month) &
                                                   (subtimeseries_o.index.year == time_period.year)]
                subtimeseries_dp = subtimeseries_d[(subtimeseries_d.index.month == time_period.month) &
                                                   (subtimeseries_d.index.year == time_period.year)]
                # subtimeseries_o[subtimeseries_o.index.weekday >= 5] = subtimeseries_o[subtimeseries_o.index.weekday >= 5] * 0
                alpha_o = subtimeseries_op / subtimeseries_op.sum() #Hallamos alpha
                alpha_d = subtimeseries_dp / subtimeseries_dp.sum() #alpha corresponde a toda la matriz de distritos y desplazamientos normalizados, cada iteración alpha es un mes diferente

                # subtimeseries_d[subtimeseries_d.index.weekday >= 5] = subtimeseries_d[subtimeseries_d.index.weekday >= 5] * 0

                up_aux = pd.DataFrame({}, index=subtimeseries_op.index) #DataFrame up_aux tendrá el mismo índice que el DataFrame subtimeseries_op.
                down_aux = pd.DataFrame({}, index=subtimeseries_dp.index)
                for stop_id in beta.stop_id:

                    up_aux[stop_id] = alpha_o[stops.loc[stop_id]] * Td.loc[time_period].iloc[0] * 1e6 * \
                                      beta[beta.stop_id == stop_id].up.iloc[0] 
                    #Estas son las formulas que calculan el número de pasajeros por estación
                    down_aux[stop_id] = alpha_d[stops.loc[stop_id]] * Td.loc[time_period].iloc[0] * 1e6 * \
                                        beta[beta.stop_id == stop_id].down.iloc[0]
                print(up_aux)  #df con los desplazamientos calculados por estación, cada iteración del ciclo corresponde a un df con un mes diferente
                up = up.append(up_aux)
                down = down.append(down_aux)
            print(up) # Muestra todos los desplazamientos calculados por estación, cada columna es una estación

        elif service == 'metro':
            for time_period in Td.index:
                subtimeseries_op = subtimeseries_o[subtimeseries_o.index.date == time_period.date()]
                subtimeseries_dp = subtimeseries_d[subtimeseries_d.index.date == time_period.date()]

                alpha_o = subtimeseries_op / subtimeseries_op.sum()
                alpha_d = subtimeseries_dp / subtimeseries_dp.sum()

                up_aux = pd.DataFrame({}, index=subtimeseries_op.index)
                down_aux = pd.DataFrame({}, index=subtimeseries_dp.index)
                for stop_id in beta.stop_id:
                    up_aux[stop_id] = alpha_o[stops.loc[stop_id]] * Td.loc[time_period].iloc[0] * \
                                      beta[beta.stop_id == stop_id].up.iloc[0]
                    down_aux[stop_id] = alpha_d[stops.loc[stop_id]] * Td.loc[time_period].iloc[0] * \
                                        beta[beta.stop_id == stop_id].down.iloc[0]
                up = up.append(up_aux)
                down = down.append(down_aux)

        return up, down